## Integrate retry into raster package and see if more files are processed overall

Environment: `rasterRetry`

packages:
```
Package            Version    Editable project location
------------------ ---------- -------------------------------
affine             2.3.1
asttokens          2.0.5
attrs              22.1.0
backcall           0.2.0
bcrypt             4.0.1
certifi            2022.9.24
cffi               1.15.1
charset-normalizer 2.1.1
click              8.1.3
click-plugins      1.1.1
cligj              0.7.2
coloraide          0.18.1
colormaps          0.3
contourpy          1.0.6
cryptography       38.0.4
cycler             0.11.0
debugpy            1.5.1
decorator          5.1.1
dill               0.3.6
entrypoints        0.4
executing          0.8.3
filelock           3.8.2
Fiona              1.8.22
fonttools          4.38.0
geopandas          0.12.2
globus-sdk         3.15.1
idna               3.4
ipykernel          6.15.2
ipython            8.7.0
jedi               0.18.1
jupyter_client     7.4.7
jupyter_core       4.11.2
kiwisolver         1.4.4
matplotlib         3.6.2
matplotlib-inline  0.1.6
morecantile        3.2.5
munch              2.5.0
nest-asyncio       1.5.5
numpy              1.24.0
packaging          22.0
pandas             1.5.2
paramiko           2.12.0
parsl              2022.12.19
parso              0.8.3
pdgraster          0.1.0      /home/jcohen/viz-raster-w-retry
pdgstaging         0.1.0
pexpect            4.8.0
pickleshare        0.7.5
Pillow             9.3.0
pip                22.3.1
prompt-toolkit     3.0.20
psutil             5.9.0
ptyprocess         0.7.0
pure-eval          0.2.2
pycparser          2.21
pydantic           1.10.2
Pygments           2.11.2
PyJWT              2.6.0
PyNaCl             1.5.0
pyparsing          3.0.9
pyproj             3.4.1
python-dateutil    2.8.2
pytz               2022.7
pyzmq              23.2.0
rasterio           1.3.4
requests           2.28.1
Rtree              0.9.7
setproctitle       1.3.2
setuptools         65.5.0
shapely            2.0.0
six                1.16.0
snuggs             1.4.7
stack-data         0.2.0
tblib              1.7.0
tornado            6.2
traitlets          5.7.1
typeguard          2.13.3
typing_extensions  4.4.0
urllib3            1.26.13
wcwidth            0.2.5
wheel              0.37.1
```

First test out what the modified code is doing by creating an MRE.

Code with retry:

```python
    def rasterize_vector(self, path):
        """
            Given a path to an output file from the viz-staging step, create a
            GeoTIFF and save it to the configured dir_geotiff directory. If the
            output geotiff already exists, it will be overwritten.
            During this process, the min and max values (and other summary
            stats) of the data arrays that comprise the GeoTIFFs for each band
            will be tracked.
            Parameters
            ----------
            path : str
                Path to the staged vector file to rasterize.
            Returns
            -------
            morecantile.Tile or None
                The tile that was rasterized or None if there was an error.
        """

        # if error occurs, retries
        for attempt in range(2):
            try:

                # Get information about the tile from the path
                tile = self.tiles.tile_from_path(path)
                bounds = self.tiles.get_bounding_box(tile)
                out_path = self.tiles.path_from_tile(tile, 'geotiff')

                # Track and log the event
                id = self.__start_tracking('geotiffs_from_vectors')
                logger.info(f'Rasterizing {path} for tile {tile} to {out_path}.')

                # Check if deduplication should be performed first
                gdf = gpd.read_file(path)
                dedup_here = self.config.deduplicate_at('raster')
                dedup_method = self.config.get_deduplication_method()
                if dedup_here and (dedup_method is not None):
                    dedup_config = self.config.get_deduplication_config(gdf)
                    dedup = dedup_method(gdf, **dedup_config)
                    gdf = dedup['keep']

                # Get properties to pass to the rasterizer
                raster_opts = self.config.get_raster_config()

                # Rasterize
                raster = Raster.from_vector(
                    vector=gdf, bounds=bounds, **raster_opts)
                raster.write(out_path)

                # Track and log the end of the event
                message = f'Rasterization for tile {tile} complete.'
                self.__end_tracking(id, raster=raster, tile=tile, message=message)
                logger.info(
                    f'Complete rasterization of tile {tile} to {out_path}.')

                return tile

            except Exception as e:
                logger.info(f'Error rasterizing {path} for tile {tile} so trying again.')
            
            else:
                break
        
        else:
            message = f'Error rasterizing {path} for tile {tile}, ran out of retries.'
            self.__end_tracking(id, tile=tile, error=e, message=message)
            return None
```

In [10]:
# create example function

def test_func(x, y):
    for attempt in range(2):
        try:
            z = x / y
            print(f'Success: {x} / {y} = {z}')
            return z
        except Exception as e:
            print(f'Error dividing {x} by {y} so trying again.')
        else:
            break

    else:
         print(f'Error dividing {x} by {y}, ran out of retries.')
         return None
    

In [11]:
# test function with expected input types
test_func(x = 9, y = 3)

Success: 9 / 3 = 3.0


3.0

In [12]:
# test function with erronous inputs
test_func(x = 'elephant', y = 'cat')

Error dividing elephant by cat so trying again.
Error dividing elephant by cat so trying again.
Error dividing elephant by cat, ran out of retries.
